# MATH 152 Lab 4 Overview: Numerical Integration

Recall that the integral of a continuous function $f(x)$ on an interval $[a,b]$ is equal to the limit $$ \int_a^b f(x)\,\mathrm dx = \lim_{N\to\infty}\sum_{i=1}^Nf(x_{i,N})\Delta x_N $$ where $x_{i,N} := a + i\Delta x_N$ and $\Delta x_N := (b-a)/N$. Therefore, for large values of $N$, we expect that the sum on the right is a good approximation to the integral of $f$: $$ \int_a^b f(x)\,\mathrm dx \approx \sum_{i=1}^Nf(x_{i,N})\Delta x_N. $$ We will implement the sum on the right and use it to approximate the integral of $f(x) = e^{-x^2}$ on $[0,2]$.

In [1]:
import sympy as sp

# approximate the integral of a continuous function f(x) on an interval [a, b]
# using a Riemann sum with N terms
def integrate(f,a,b,N):
    dx=(b-a)/N
    integral = 0
    for i in range(1,N+1): # range(m,n) = [m, m+1, ..., n-1]
        xi = a + i*dx
        integral = integral + f(xi)*dx
    return integral

# test our implementation with f(x) = e^{-x^2}
def f(x):
    return sp.exp(-x**2)

# have SymPy compute the exact value for comparison
x = sp.symbols('x')
integral = sp.integrate(sp.exp(-x**2),(x,0,2))
print("The true value is",integral,"~",integral.evalf())
print()

# compute approximations with increasing values of N
for N in [int(2**n) for n in range(10)]: # [1, 2, 4, 8, 16, ...]
    exact = integral.evalf()
    approx = integrate(f,0,2,N)
    error = sp.Abs(approx - exact)/sp.Abs(exact)
    print("The approximation with\t",N,"\tsubintervals is\t",approx,"\t with error\t",error)

The true value is sqrt(pi)*erf(2)/2 ~ 0.882081390762422

The approximation with	 1 	subintervals is	 0.0366312777774684 	 with error	 0.958471771243461
The approximation with	 2 	subintervals is	 0.386195080060176 	 with error	 0.562177499599701
The approximation with	 4 	subintervals is	 0.635197543846723 	 with error	 0.279887830648265
The approximation with	 8 	subintervals is	 0.758993246193225 	 with error	 0.139542842483964
The approximation with	 16 	subintervals is	 0.820630972696323 	 with error	 0.0696652471185048
The approximation with	 32 	subintervals is	 0.851379921516484 	 with error	 0.0348057101844093
The approximation with	 64 	subintervals is	 0.866736611468431 	 with error	 0.0173961036415561
The approximation with	 128 	subintervals is	 0.874410491221382 	 with error	 0.00869636251413197
The approximation with	 256 	subintervals is	 0.878246313597994 	 with error	 0.00434775884015967
The approximation with	 512 	subintervals is	 0.880163945336707 	 with error	 0.00

We make the following observations:
1. SymPy tells us that the exact value of $$ \int_0^2 e^{-x^2}\,\mathrm dx = \frac{\sqrt{\pi}}{2}\operatorname{erf}(2), $$ where, by definition, $$ \operatorname{erf}(x) := \frac{2}{\sqrt{\pi}}\int_0^x e^{-t^2}\,\mathrm dt. $$
In other words, there is no way to actually find an antiderivative of $e^{-x^2}$ and use it to compute a useful representation of the exact value of the integral.
We _must_ use numerical integration to compute a number that approximates this integral.

2. Our numerical routine `integrate` does indeed let us approximate the integral, but it is not very accurate, and it is slow.
We have to use $N \geq 16$ to get at least 1 significant digit of accuracy and $N \geq 512$ to get at least 2 siginficant digits of accuracy.
In modern practice, most computations require at least 6 siginficant digits of accuracy to be considered even moderately accurate, which roughly corresponds to having a relative error smaller than $10^{-6}$.
As it stands, our `integrate` routine is too slow and too innacurate to be practically useful.

3. In contrast, the `.evalf()` method from SymPy gives us the numerical approximation to the integral with 15 significant digits of accuracy almost immediately.
Their numerical algorithm is clearly better than the one we created.
Therefore, in practice, it is generally better to try using tools that others have written before trying to implement things yourself.

We will now showcase some other tools that are out there in the Python ecosystem to perform numerical integration.
The primary one that we will showcase is the `scipy.integrate` subpackage from the `scipy` package https://docs.scipy.org/doc/scipy/tutorial/integrate.html.

There are a lot of different integration routines provided by `scipy.integrate`.
We will show one particular one called the Composite Simpson's rule https://en.wikipedia.org/wiki/Simpson%27s_rule#Composite_Simpson's_rule_for_irregularly_spaced_data.
The particular details of this numerical integration method are beyond the scope of this course, but one should be aware that there are more sophisticated techniques out there that give faster and more accurate results.

We will now use the Composite Simpson's rule to estimate the integral of $f(x) = e^{-x^2}$ on $[0,2]$ as above.
We follow the documentation for this function https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.simpson.html#scipy.integrate.simpson and provide what it needs to compute the integral.

In [ ]:
from scipy.integrate import simpson

# integrate a function f(x) on an interval [a,b] using the composite Simpson's
# rule routine from SciPy with N+1 samples
def integrate_simpson(f,a,b,N):
    xs = [a + i*(b-a)/N for i in range(N+1)]
    ys = [f(x) for x in xs]
    return simpson(ys, x=xs)

# compute approximations with Simpson's rule with increasing values of N and
# compare with our naive integration method
exact_integral = integral.evalf()
print('Exact\t'+str(exact_integral))
print()
print('N\tNaive\t\t\tError\t\t\tSimpson\t\t\tError')
for N in [int(2**n) for n in range(10)]: # [1, 2, 4, 8, 16, ...]
    naive_integral = integrate(f,0,2,N)
    naive_error = sp.Abs(exact_integral - naive_integral) / sp.Abs(exact_integral)

    simpson_integral = integrate_simpson(f,0,2,N)
    simpson_error = sp.Abs(exact_integral - simpson_integral) / sp.Abs(exact_integral)

    print(str(N)+'\t'+str(naive_integral)+'\t'+str(naive_error)+'\t'+str(simpson_integral)+'\t'+str(simpson_error))

Exact	0.882081390762422

N	Naive			Error			Simpson			Error
1	0.0366312777774684	0.958471771243461	1.01831563888873	0.154446346508409
2	0.386195080060176	0.562177499599701	0.829944467858168	0.0591067031344915
4	0.635197543846723	0.279887830648265	0.881812425294116	0.000304921372474588
8	0.758993246193225	0.139542842483964	0.882065510401332	1.80032832075607e-5
16	0.820630972696323	0.0696652471185048	0.882080396576992	1.12709035728636e-6
32	0.851379921516484	0.0348057101844093	0.882081328646356	7.04198796765421e-8
64	0.866736611468431	0.0173961036415561	0.882081386880655	4.40069004690365e-9
128	0.874410491221382	0.00869636251413197	0.882081390519820	2.75033633065098e-10
256	0.878246313597994	0.00434775884015967	0.882081390747259	1.71893739380475e-11
512	0.880163945336707	0.00217377381021172	0.882081390761474	1.07437520363161e-12


Now we see that the Compsite Simpson rule gives us a more accurate answer much faster.
We can achieve at least 6 digits of accuracy with only $N \geq 32$, while our naive implementation only achieves an accuracy of 1 significant digit.